In [1]:
import pandas as pd
import os
from glob import glob
import reports
pd.options.display.float_format = '{:.2f}'.format

In [78]:
no_pt_filepaths_format = '/media/dmlab/My Passport/DATA/cross-domain/finetune_{}/source=*_post=None_target=*/classification_report.csv'
pt_raw_filepaths_format = '/media/dmlab/My Passport/DATA/cross-domain/finetune_{}/source=*_post=*-raw_target=*/classification_report.csv'
pt_random_filepaths_format = '/media/dmlab/My Passport/DATA/cross-domain/finetune_{}/source=*_post=*-random_target=*/classification_report.csv'
pt_keyword_filepaths_format = '/media/dmlab/My Passport/DATA/cross-domain/finetune_{}/source=*_post=*-keyword_target=*/classification_report.csv'
kfold_num = 1

In [79]:
def get_average_classification_report(reports):
    for i, report in enumerate(reports):
        report.set_index(report.columns[0], inplace=True)
        if i == 0: 
            report_avg = report
        else: 
            report_avg = report_avg.add(report)
    report_avg = report_avg / len(reports)
    return report_avg

def show_result(filepaths_format):
    kfold_reports = []
    for i in range(0, kfold_num):
        filepaths = glob(filepaths_format.format(i))

        records = []
        for filepath in filepaths:
            source, post, target = os.path.basename(os.path.dirname(filepath)).split('_')
            source, target = source.replace('source=',''), target.replace('target=', '')

            df = pd.read_csv(filepath)
            df.set_index('Unnamed: 0', inplace=True)
            acc = 100*df.filter(items = ['accuracy'], axis=0)['f1-score'].values[0]

            records.append(('{}->{}'.format(source, target), float(acc)))

        result_df = pd.DataFrame(records, columns = ['source->target', '{}_accuracy'])
        kfold_reports.append(result_df)

    result_df = get_average_classification_report(kfold_reports)
    result_df.loc['Average'] = result_df.mean()
    return result_df

	• Raw 소스, Raw target 테스트

In [80]:
no_pt_df = show_result(no_pt_filepaths_format)
no_pt_df.columns = ['no_pt']

pt_raw = show_result(pt_raw_filepaths_format)
pt_raw.columns = ['RAW']

pt_random = show_result(pt_random_filepaths_format)
pt_random.columns = ['RANDOM']

pt_keyword = show_result(pt_keyword_filepaths_format)
pt_keyword.columns = ['KEYWORD']

result = pd.concat([no_pt_df, pt_raw, pt_random, pt_keyword], axis=1)
result

,no_pt,RAW,RANDOM,KEYWORD
books->dvd,88.31,90.59,89.93,90.34
books->electronics,87.86,90.67,88.27,89.50
books->kitchen,88.87,90.79,90.64,91.05
electronics->books,84.81,89.30,90.06,89.30
electronics->dvd,85.36,88.66,89.37,87.75
electronics->kitchen,89.73,92.36,91.55,92.11
kitchen->books,89.05,89.96,89.76,90.67
kitchen->dvd,85.46,88.10,87.60,87.39
kitchen->electronics,89.14,90.21,91.53,90.97
dvd->books,86.33,90.97,91.62,91.12
